<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-08-04
v0.0.2, 2022-01-16
-->
[Home](../index.ipynb) / Erste Schritte mit MicroPython
***
<span style="font-size:20pt;">Erste Schritte mit MicroPython</span>

---
# Lösungsvorschläge

1. Lassen Sie die interne LED N-mal mit der Blinkperiodendauer T blinken, wobei die Leuchtzeit D der LED in Prozent der Periodendauer T angegeben werden soll.  
   Starten Sie mit 5-mal blinken, der Periodendauer 1 s und der Leuchtzeit 10%.

In [125]:
%serialconnect --port=COM3 --baud=115200 # für Windows
# %serialconnect # für Linux

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino, NodeMCU: extern 14

N = 5     # N-mal
T = 1000  # mit der Blinkperiodendauer T (ms) blinken 
D = 10    # mit der Leuchtzeit D = 10% der Blinkperiodendauer T

tOn  = int(T/100*D)
tOff = T-tOn

for i in range( N ):
    pin.on()
    time.sleep_ms( tOn )
    pin.off()
    time.sleep_ms( tOff )

print( "\nDone." )

.DONE.


2. Probieren Sie für N=100, T=20 jeweils D=5, 20 und 100

Im der Zelle oben `N`, `T` und `D` passend setzten

3. Ändern Sie D periodisch von 0 bis 100 und wieder zurück zu 0 in 5-er Schritten um die LED 5-mal puliseren zu lassen.

In [2]:
%serialconnect --port=COM3 --baud=115200 # für Windows
#%serialconnect # für Linux

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino, NodeMCU: extern 14

N = 5
T = 20

for i in range( N ):
    for D in range( 0, 101, 5 ) :
        tOn  = int(T/100*D)
        tOff = T-tOn
        pin.on()
        time.sleep_ms( tOn )
        pin.off()
        time.sleep_ms( tOff )
        
    for D in range( 100, 0, -5 ) :
        tOn  = int(T/100*D)
        tOff = T-tOn
        pin.on()
        time.sleep_ms( tOn )
        pin.off()
        time.sleep_ms( tOff )
        
print( "\nDone." )

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.

Done.


4. Lassen Sie die LED 5 mal im 1s Takt pulsieren.

In [9]:
%serialconnect --port=COM3 --baud=115200 # für Windows
#%serialconnect # für Linux

from machine import Pin, PWM
import time

pin = PWM( Pin(25, Pin.OUT), freq=1000, duty=0) # 25 HelTec, 13 Croduino, NodeMCU: extern 14

N = 5

for i in range( N ):
    for iDuty in range( 0, 1000, 2 ):
        pin.duty(iDuty)
        time.sleep( 0.001 )

    for iDuty in range( 1001, 0, -2 ):
        pin.duty(iDuty)
        time.sleep( 0.001 )
        
pin.deinit() # falls der Pin im selbem Porgramm noch mal als on/off- Pin verwendet werden soll.
    
print( "\nDone." )

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
.
Done.


5. Erstellen Sie eine animierte analoge Uhr mit Ziffernblatt (Kreis mit 12 Stunden-Striche) und Zeigern (Stunde, Minute, Sekunde).

In [11]:
%serialconnect --port=COM4 --baud=115200  
#%serialconnect

import machine
import display
import math

gSeconds = 1800

def timerInterruptHandler(timer):
    global gSeconds
    gSeconds += 1

timer = machine.Timer(1)

timer.init(period=1000, mode=machine.Timer.PERIODIC, callback=timerInterruptHandler)


gSecondsLast = 0
display = display.Display()
display.setCenter( 64, 32 )

try:
    while True:
        if gSeconds != gSecondsLast :
            #state = machine.disable_irq()
            #interruptCounter = interruptCounter-1
            #machine.enable_irq(state)
            gSecondsLast = gSeconds

            display.clear()
            display.circle( 0,0,6)
            for iH in range( 12 ) :
                display.lineP( 0,0,math.pi/6*iH, 30, 32 )
            display.fillCircle( 0,0,4)
            s = gSeconds%60
            m = int(gSeconds/60)%60
            h = int(gSeconds/3600)%24
            display.lineP( 0,0,math.pi/ 6*h+math.pi/360*+math.pi/360*m-math.pi/2, 0, 20 )
            display.lineP( 0,0,math.pi/30*m+math.pi/1800*s-math.pi/2, 0, 25 )
            display.lineP( 0,0,math.pi/30*s-math.pi/2, 0, 32 )
            display.show()
except KeyboardInterrupt:
    timer.deinit()
    
print( "Interrupted by user!" )
print( "\nDone." )

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
.

*** Sending Ctrl-C

Interrupted by user

Done.


6. Schreiben Sie ein Programm, das Ihnen den Vorwiderstand bei gegebener Spannung für eine rote LED berechnet und den passenden Widerstand der [E6-er Reihe](https://www.elektronik-kompendium.de/sites/bau/1109071.htm) ausgibt.


In [86]:
%serialconnect --port=COM4 --baud=115200  
#%serialconnect
# E6-er Reihe: 1, 1.5, 2.2, 3.3, 4.7, 6.8

from math import log, floor

u    = 3.3   # Maximale positive Spannung am Pin.
uLED = 1.6   # Schwellenspannung der roten LED
iMax = 10E-3 # Maximal zulässiger Strom druch den Pin (0.6E-3 fuer Esp8622 Croduino Nova)


r = (u-uLED)/iMax

if r > 0 :
    iPotenz = floor( log( r, 10 )  )
    rNorm = r/10**iPotenz
    if   rNorm <= 1   : rE6 =  1
    elif rNorm <= 1.5 : rE6 =  1.5
    elif rNorm <= 2.2 : rE6 =  2.2
    elif rNorm <= 3.3 : rE6 =  3.3
    elif rNorm <= 4.7 : rE6 =  4.7
    elif rNorm <= 6.8 : rE6 =  6.8
    else              : rE6 = 10
    
    rE6 = int( rE6 * 10**iPotenz )
    
    print( "Spannung U = {} V, Schwellenspannung U_LED = {} V, maximaler Strom iMax = {} A".format( u, uLED, iMax ) )
    print( "==> R = {} Ohm ==> R_E6 = {} Ohm".format( r, rE6 ) )
else:
    print( "Widerstandswert {} <= 0 ist nicht möglich".format( r ) )


Spannung U = 3.3 V, Schwellenspannung U_LED = 1.6 V, maximaler Strom iMax = 0.01 A
==> R = 170.0 Ohm ==> R_E6 = 220 Ohm


7. Schreiben Sie die Programme der internen LED für die externe LED.

Nicht viel zu tun: Pin-Nummer der internen LED durch z.B. `14` ersetzten.

8. Immer wenn die externe LED an ist, soll die interne aus sein und umgekehrt

Am einachsten so: LED an den Pin der internen LED über passenden Vorwiderstand auf 3V3 schalten!

9. Am Display soll der Zustand der externen LED passen angezeigt werden.
```python
pin.on() # pin.off()
```
ersetzen durch
```python
pin.on()
display.fill_rect(40, 10, 25, 7, 0)
display.text( "LED on" ) # display.text( "LED off" )
display.show()
```

In [104]:
#%serialconnect --port=COM4 --baud=115200  
%serialconnect

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino, NodeMCU: extern 14

import display
display = display.Display()

import time

for i in range(10):
    display.fill_rect(40, 10, 25, 7, 0)
    
    if i%2==0:
        pin.on()
        display.text( "LED on", 10, 10 )
    else:
        pin.off()
        display.text( "LED off", 10, 10  )
        
    display.show()
    
    time.sleep_ms(500)

display.text( "Done.", 10, 30  )       
display.show()
    
print( "\nDone." )

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
.
Done.


10. Das Programm soll auf dem Display die gefundenen Stationen mit den zur Verfügung stehenden Informationen ausgeben und alle 5 Sekunden updaten.

In [139]:
#%serialconnect --port=COM4 --baud=115200  
%serialconnect
import network
import time
import socket
import binascii

import display
display = display.Display()

wlan = network.WLAN(network.STA_IF) # create station interface
wlan.active(True)                   # activate the interface

for iCount in range( 5 ) :
    print( "\nScan...\n=========================" )
    
    nets = wlan.scan()
    
    iIndex = 0
    display.clear()
    
    for net in nets: # net = (ssid, bssid, channel, RSSI, authmode, hidden)
        strSSID      = net[0] #.decode('UTF-8')
        strBSSID     = binascii.hexlify(net[1]).decode('UTF-8') # Hardware address of an access point (in binary form)
        strChannel   = str( net[2] )
        strRSSI      = "{}% ({} dB)".format( round( net[3]/-120*100 ), net[3] ) # Received Signal Strength Indicator (RSSI): in dB from -120 to 0.

        if   net[4] == 0 : strAuthmode = "open"
        elif net[4] == 1 : strAuthmode = "WEP"
        elif net[4] == 2 : strAuthmode = "WPA-PSK"
        elif net[4] == 3 : strAuthmode = "WPA2-PSK"
        elif net[4] == 4 : strAuthmode = "WPA/WPA2-PSK"
        else             : strAuthmode = "unknown"

        strVisibility = "visible" if net[5] == 0 else "hidden"

        print( "ssid    : {}".format( strSSID    ) )
        print( "bssid   : {}".format( strBSSID   ) ) 
        print( "channel : {}".format( strChannel ) )

        print( "RSSI    : {}".format( strRSSI ) ) 
        print( "authmode: {}".format( strAuthmode   ) )
        print( "hidden  : {}".format( strVisibility ) )

        print( "-------------------------" )

        display.text( "{}: {}".format( iIndex+1, strSSID ), 0, iIndex*10 )
        iIndex += 1
    display.show()
    
    time.sleep(3)
        
    
print( "\nDONE." )

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.

Scan...
ssid    : b'FRITZ!Box Gast'
bssid   : 3a10d5616217
channel : 1
RSSI    : 57% (-68 dB)
authmode: WPA2-PSK
hidden  : visible
-------------------------
ssid    : b'FRITZ!Box'
bssid   : 3810d5616217
channel : 1
RSSI    : 58% (-69 dB)
authmode: WPA2-PSK
hidden  : visible
-------------------------
ssid    : b'FRITZ!Box 7520 TQ'
bssid   : 74427faa59e1
channel : 11
RSSI    : 62% (-75 dB)
authmode: WPA2-PSK
hidden  : visible
-------------------------
ssid    : b'meins_7490'
bssid   : 3810d5ab5302
channel : 11
RSSI    : 73% (-88 dB)
authmode: WPA2-PSK
hidden  : visible
-------------------------
ssid    : b''
bssid   : 969a4a94aa68
channel : 4
RSSI    : 77% (-92 dB)
authmode: WPA2-PSK
hidden  : visible
-------------------------
ssid    : b'TP-Link_AA68'
bssid   : 909a4a94aa68
channel : 4
RSSI    : 78% (-93 dB)
authmode: WPA/WPA2-PSK
hidden  : visible
-------------------------
.
Scan...
ssid    : b'FRITZ!Box 7520 TQ'
bssid   : 74427f